In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.filters import threshold_local
from PIL import Image

In [21]:

def opencv_resize(image, ratio):
    width = int(image.shape[1] * ratio)
    height = int(image.shape[0] * ratio)
    dim = (width, height)
    return cv2.resize(image, dim, interpolation = cv2.INTER_AREA)

def plot_rgb(image):
    plt.figure(figsize=(16,10))
    return plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

def plot_gray(image):
    plt.figure(figsize=(16,10))
    return plt.imshow(image, cmap='Greys_r')
# approximate the contour by a more primitive polygon shape
def approximate_contour(contour):
    peri = cv2.arcLength(contour, True)
    return cv2.approxPolyDP(contour, 0.032 * peri, True)

def get_receipt_contour(contours):    
    for c in contours:
        approx = approximate_contour(c)
        print(len(approx))
        # four points = rectangle
        if len(approx) == 4:
            print(f"taken:{len(approx)}")
            return approx


def wrap_perspective(img, rect):
    # unpack rectangle points: top left, top right, bottom right, bottom left
    (tl, tr, br, bl) = rect
    # compute the width of the new image
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    # compute the height of the new image
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    # take the maximum of the width and height values to reach
    # our final dimensions
    maxWidth = max(int(widthA), int(widthB))
    maxHeight = max(int(heightA), int(heightB))
    # destination points which will be used to map the screen to a "scanned" view
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")
    # calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(rect, dst)
    # warp the perspective to grab the screen
    return cv2.warpPerspective(img, M, (maxWidth, maxHeight))

def bw_scanner(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    T = threshold_local(gray, 21, offset = 5, method = "gaussian")
    return (gray > T).astype("uint8") * 255


In [ ]:

# path = r"D:\photos\large reciept images\1092-receipt.jpg"
path = r"D:\photos\Image_202407042318302.jpg"
# path = r"D:\photos\Image_202407042318301.jpg"
# path = r"D:\photos\Image_202407042318222.jpg"
path = r"D:\photos\large reciept images\1079-receipt.jpg"
path = r"D:\photos\large reciept images\1083-receipt.jpg"
path = r"D:\photos\large reciept images\1000-receipt.jpg"
path = r"D:\photos\large reciept images\1106-receipt.jpg"
path = r"D:\photos\large reciept images\1094-receipt.jpg"
# Add a border to the image
def add_border(image, border_size=2):
    color = [0, 0, 0]  # White border for visibility
    # color = [255, 255, 255]
    return cv2.copyMakeBorder(image, border_size, border_size, 0, 0, cv2.BORDER_CONSTANT, value=color)

# Load and add border
image = cv2.imread(path)
# height, _ = image.shape[:2]
# margin_size = 100
# image = image[margin_size:height - margin_size, :]
# image = add_border(image)
original = image.copy()
# Downscale image 
resize_ratio = 500 / image.shape[0]

image = opencv_resize(image, resize_ratio)
# Convert to grayscale 
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
plot_gray(gray)
# Get rid of noise 
blurred = cv2.GaussianBlur(gray, (7, 7), 0)
# Detect white regions
rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
dilated = cv2.dilate(blurred, rectKernel, iterations=4)
plot_gray(dilated)
dilated = add_border(dilated)

edged = cv2.Canny(dilated, 75, 200, apertureSize=3)
# plot_gray(edged)

# Apply morphological operations to close gaps between contours
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))  # Adjust kernel size as needed
closed = cv2.morphologyEx(edged, cv2.MORPH_CLOSE, kernel)
plot_gray(closed)
# Detect all contours 
contours, hierarchy = cv2.findContours(closed, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
print(len(contours))
def contour_to_rect(contour):
    pts = contour.reshape(4, 2)
    rect = np.zeros((4, 2), dtype = "float32")
    # top-left point has the smallest sum
    # bottom-right has the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    # compute the difference between the points:
    # the top-right will have the minumum difference 
    # the bottom-left will have the maximum difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect / resize_ratio

# Get 10 largest contours
filtered_contours = [c for c in contours if cv2.contourArea(c) > 1000]
largest_contours = sorted(filtered_contours, key = cv2.contourArea, reverse = True)[:10]
receipt_contour = get_receipt_contour(largest_contours)
image_with_receipt_contour = cv2.drawContours(image.copy(), [receipt_contour], -1, (0, 255, 0), 2)
plot_rgb(image_with_receipt_contour)

In [ ]:
original = wrap_perspective(original.copy(), contour_to_rect(receipt_contour))
plt.figure(figsize=(16,10))
plt.imshow(original)
greyscale_output = bw_scanner(original)

output = Image.fromarray(original)
# output.save('result.png')
plot_gray(greyscale_output)
# greyscale_output